# Memoir Close Reading

## Journal de Goncourt

words/phrases to investigate:

(need French translations) the left, the _ left, suppression, mass rising, mobocracy, insurrection, insurgents, National Guards, Republicanism, reg flag, crowd, mob, gang, violence, friendliness, Prussians, Bellevillans, murders, cannon, cannonade, noise, aggressive, drunk, drunkenness, intoxicated, Federals, barricade, altercation, middle-class, upper-class, shout, dispute, fight, corpse, death, wounded, burning, burn, blood, bullets, shells

places:

College de France, Montmartre, Courberoie, Hotel de Ville, Issy, Neuilly, Arc de l'Etoile, Auteuil, Pepiniere barracks, Opera House, Jardin des Plantes, Faubourg Saint-Germain, Monceau, Batignolles, Tuileries, Passy, Palais Royal, Tuileries, Palais de Justice, Preferecture of Police

people:

Jules Valles, Assi, Clement Thomas, Lecomte, Archbishop, Mont-Valerian, MacMahon

timeline of events and selected quotes:

70.09.25: notice posted outside the college de france announcing the formation of the Commune. demands the suppression of the Prefecture of POlice

70.09.30: "liberty, equality, fraternity" appears

70.10.28: National Guards recruiting on a platform with stairs "draped in the red cloth used by wine merchants"

70.10.31: National Guards shouting "Vive la Commune". "The Government is upset, the Commune established, and the list of the man who has been dictating is to be confirmed by universal suffrage." "Vociferous and gesticulating mobocracy" The date "of a revolution." Except - he doesn't mention the Commune again for a while.

71.03.18: "fighting going on at Montmartre" "triumphing insurrection takes possession of Paris." "Hotel de Ville... orators are speaking of putting traitors to death" "Clement Thomas and Lecomte had been shot"

71.03.19: "Army is in full retreat upon Versailles"

71.03.20: Complains about no "National Guard of the Middle Classes"

71.03.31: Jules Valles, Assi (I forget context)

71.04.02: "Cannonade in the direction of Courberoie. Civil War has commenced!" "... hypocritical murders." "... the Bellevillans have been defeated."

71.04.03: More cannonade, mentions becoming accustomed to cannonade.

71.04.04: "Drunkenness of the National Guards is becoming aggressive to passers-by"

71.04.05: "government of the Committee seems to be at an end; yet the cannonade lasted all day around the fort at Issy, where one sees the largest red flag waving"

71.04.07: "it's quite likely the archbishop will be shot tonight". referencing plan to confiscate weapons: "petty preparations for a reign of terror" 

71.04.09: weapons confiscated from citizens

71.04.14: incoming draft mentioned. 

71.04.15: "I am more frightened than I ever was in the time of the Prussians"

71.04.21: Draft

Time skip - journal says the entries are almost exclusively about bombing and hatred of the Communards

71.05.22: barricade "guarded by a few men who look the reverse of energetic". Versailles at Pepiniere barracks. "National Guards and middle-class men are having altercations all over the place." "there is not a single member of the Commune along the whole line of boulevards" "shouts, disputes, and fights" "everybody whom the National Guard finds in the streets is forcibly enlisted", "hear all night the voices of intoxicated National Guards, shoating hoarsely at every passer-by. It was a day when I at last fell into a sleep broken by explosions and nightmares."

71.05.23: Versailles occupy Faubourg Saint-Germain, Monceau, and Batignolles "immense battlefield... engagements which were taking place from Jardin des Plantes as far as Montmartre" "One sees a corpse with a bleeding head, which four men carry by the arms and legs like a packet of dirty linen, taking it from door to door, none of which will open"

71.05.28: He surveys the damage done to Paris. Sees 26 prisoners walking to their execution. Executioners exit "like drunken men"

71.05.29: "one once more feels the movement of Paris coming to life again"

71.05.30: "from time to time we hear frightful noises - houses collapsing, or volleys"




        







In [9]:
import re

# Delete pre-text data
# Delete page headers (eg JOURNAL DE LA COMMUNE 49)
## eg code: tmp[tmp.find("JOURNAL DE LA COMMUNE"):]
# Delete \n
# Split based on chapters

# Elie
f = open('memoirs_complete\\elie.txt', 'r')
tmp = f.read()
tmp = tmp[tmp.find("JOURNAL DE LA COMMUNE"):]
tmp = re.sub(r"[0-50]\sJOURNAL\sDE\sLA\sCOMMUNE", "", tmp)
tmp = re.sub(r"JOURNAL\sDE\sLA\sCOMMUNE\s[1-50]", "", tmp)
print(tmp)

JOURNAL DE LA COMMUNE

 

Paris, 20 mars 1871.

Préparé de longue main par M. Thiers et ses complices,

le coup d'Etat qu'on sentait confusément avancer dans
l'ombre a enfin éclaté et avorté.
On a trouvé dans les Ministères des télégrammes échangés
entre Thiers et Favre, entre le Ministre de la guerre et le
Préfet de police, entre l'honorable Jules Favre et l'hono-
rable Jules Ferry. Nous en donnons quelques extraits.
D'abord lu série Thiers à Jules Favre qui, malheureuse-
ment, ne s étend que du 4 au 6 mars... : « Je vous expédie
le Général de Paladines... et trois divisions représentant
30,000 hommes sont en marche. Mais les troupes, même
en chemin de fer, ne peuvent pas aller aussi vite que vous
le supposez. Croyez qu'en fait de choses pareilles, rien ne
sera négligé... Nous allons vous renvoyer aussi Picard et
un ou deux de vos collèques.. Il n'est pas possible que la
garde nationale n'intervienne pas. Si elle ne je fait pas,
nous le ferons.

Thiers à Vinoy et à Ministre de la guer

In [218]:
import pandas as pd
cleaned_data = []
for row in df.iterrows():
    tmp_num_inventory = ''
    tmp_category = ''
    tmp_showcase = ''
    tmp_manufacturer = ''
    tmp_country = ''
    tmp_date = ''
    tmp_dimensions = ''
    for item in row[1]['object_card'].split('\n'):
        if item.startswith("N° d'inventaire "):
            tmp_num_inventory = item.replace("N° d'inventaire ", "")
        if item.startswith("Catégorie "):
            tmp_category = item.replace("Catégorie ", "")
        if item.startswith("Vitrine "):
            tmp_showcase = item.replace("Vitrine ", "")
        if item.startswith("Constructeur "):
            tmp_manufacturer = item.replace("Constructeur ", "")
        if item.startswith("Pays "):
            tmp_country = item.replace("Pays ", "")
        if item.startswith("Datation "):
            tmp_date = item.replace("Datation ", "")
        if item.startswith("Dimensions "):
            tmp_dimensions = item.replace("Dimensions ", "")
    tmp_inscriptions = row[1]['accordion_content'].replace("Inscriptions","")
    cleaned_data.append([tmp_num_inventory, tmp_category, tmp_showcase, tmp_manufacturer, tmp_country, tmp_date, tmp_dimensions, tmp_inscriptions])
df_scraped = pd.DataFrame(cleaned_data, columns = ['num','cat','showcase','manu','country','date','dimensions','inscriptions'])


In [219]:
df_scraped_forclustering = df_scraped[['num','showcase', 'manu', 'country', 'date', 'dimensions']]

In [220]:
# Group these
df_scraped_forclustering['manu'].value_counts()

manu
Inconnu                                      178
Hartmann & Braun, Frankfurt a. M.             25
Trüb, Täuber, Zürich                          19
Philips                                       15
Siemens & Halske, Berlin                      14
                                            ... 
SABA (Schwarzwälder-Apparate-Bau Anstalt)      1
Hartmann & Braun , Frankfurt a. M.             1
A.O.I.C. ?                                     1
Herst. Schott & Gen. , Jena                    1
Trüb, Täuber & Co Hombrechtikon-Zürich         1
Name: count, Length: 570, dtype: int64

In [221]:
# Date splitting
import numpy as np
df_scraped_forclustering['date_split'] = df_scraped_forclustering['date'].str.split("-", n = 1)
df_scraped_forclustering[['date_start_n','date_end_n']] = pd.DataFrame(df_scraped_forclustering['date_split'].tolist(), index=df_scraped_forclustering.index)
df_scraped_forclustering['date_end_n'] = df_scraped_forclustering['date_end_n'].str.split().str[0]
df_scraped_forclustering['date_start_n'] = df_scraped_forclustering['date_start_n'].str.strip()
df_scraped_forclustering['date_end_n'] = df_scraped_forclustering['date_end_n'].str.strip()
#df_scraped_forclustering.loc[~df_scraped_forclustering['date_end_n'].astype(str).str.isdigit(), 'date_end_n'].tolist()
df_scraped_forclustering['date_start_n'] = pd.to_numeric(df_scraped_forclustering['date_start_n'])
df_scraped_forclustering['date_end_n'] = pd.to_numeric(df_scraped_forclustering['date_end_n'], errors='coerce')
df_scraped_forclustering['date_end_n'] = np.where(np.isnan(df_scraped_forclustering['date_end_n']), df_scraped_forclustering['date_start_n'], df_scraped_forclustering['date_end_n'])
df_scraped_forclustering = df_scraped_forclustering.drop(columns=['date_split', 'date'])
df_scraped_forclustering = df_scraped_forclustering.astype({'date_start_n': 'float'})
df_scraped_forclustering = df_scraped_forclustering.astype({'date_end_n':'float'})

C:\Users\HP\AppData\Local\Temp\ipykernel_4312\2084500211.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scraped_forclustering['date_split'] = df_scraped_forclustering['date'].str.split("-", n = 1)


In [222]:
# Set N/A country, showcase, manu to instrument-specific value (impossible to match in Gower's distance)
# Potentially drop showcase? It's in effect supervised clustering with low coverage
df_scraped_forclustering['country'] = np.where(df_scraped_forclustering['country'] == "——", df_scraped_forclustering['num'], df_scraped_forclustering['country'])
df_scraped_forclustering['showcase'] = np.where(df_scraped_forclustering['showcase'] == "Cet instrument n'est pas en exposition", df_scraped_forclustering['num'], df_scraped_forclustering['showcase'])
df_scraped_forclustering['manu'] = np.where(df_scraped_forclustering['manu'] == "Inconnu", df_scraped_forclustering['num'], df_scraped_forclustering['manu'])


In [223]:
df_scraped_forclustering = df_scraped_forclustering[['showcase', 'manu', 'country', 'date_start_n', 'date_end_n']]
print(df_scraped_forclustering.isna().any(axis=1).sum())
df_scraped_for_clustering = df_scraped_forclustering.dropna()
print(len(df_scraped_forclustering))

3
1033


In [227]:
#https://towardsdatascience.com/clustering-on-mixed-data-types-5fe226f9d9ca
#https://towardsdatascience.com/clustering-on-mixed-data-types-5fe226f9d9ca
import gower
# Prepare data a bit more
distance_matrix = gower.gower_matrix(df_scraped_forclustering)
distance_matrix
np.argwhere(np.isnan(distance_matrix))

array([[   0,  869],
       [   0,  949],
       [   0, 1001],
       ...,
       [1032,  869],
       [1032,  949],
       [1032, 1001]], dtype=int64)

In [229]:
df_scraped_forclustering[1032]
distance_matrix[1032,1001]

KeyError: 1032

In [214]:
from sklearn.cluster import DBSCAN
dbscan_cluster = DBSCAN(eps=0.3, min_samples=2,metric='precomputed')
dbscan_cluster.fit(distance_matrix)
df_scraped_forclustering['cluster'] = dbscan_cluster.labels_

ValueError: Input X contains NaN.
DBSCAN does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [135]:
df_scraped_forclustering['cluster'].value_counts()

cluster
-1     999
 12      4
 1       3
 6       3
 0       2
 2       2
 3       2
 4       2
 5       2
 7       2
 8       2
 9       2
 10      2
 11      2
 13      2
 14      2
Name: count, dtype: int64

In [176]:
df_scraped_forclustering

,num,showcase,manu,country,dimensions,date_start_n,date_end_n
0,603.0003,603.0003,"Radiguet & Massiot, Paris",FR,Socle bois 365 x 220 mm2 h = 430 mm,1900.0,1920.0
1,603.0004,603.0004,"Steinheil, München",DE,H 400 mm ø couvercle 235 mm,1870.0,1914.0
2,603.0005,603.0005,"Trüb, Täuber & Co., Zürich",CH,ø frontal 207 mm,1911.0,1934.0
3,603.0006,603.0006,603.0006,603.0006,650 x 280 x 405 mm3,1880.0,1930.0
4,603.0007,603.0007,"J. Carpentier, Paris",FR,"Ø 188mm, h = 70 mm (sans les bornes) boîte 212...",1900.0,1921.0
...,...,...,...,...,...,...,...
1028,603.1029,603.1029,S.I.F. (Société Indépendante de T.S.F.),FR,Htot = 115 mm; ø 66 mm,1922.0,1926.0
1029,603.1030,603.1030,"Haag-Streit, Berne",CH,"Boîte : 64,5 x 60,5 x 18 cm",1923.0,1933.0
1030,603.1031,603.1031,"F. Pellin, Paris",FR,,1927.0,1941.0
1031,603.1032,603.1032,"PAR (Princeton Applied Research, Princeton, Ne...",USA,Module NIM 4/12 env. 138 x 300 x 225 mm,1969.0,1971.0
